Compare House TCJA total change in liability when using default weights vs. those provided by Ernie Tedeschi. 

Uses behavior file provided by Matt Jensen.

In [1]:
from taxcalc import *
from taxcalc.utils import *

In [2]:
recs_orig = Records.cps_constructor()

In [3]:
recs_alt = Records(data=os.path.join(Records.CUR_PATH, 'cps.csv.gz'),
                   weights='cps_weights_alt.csv',
                   adjust_ratios=None,
                   start_year=Records.CPSCSV_YEAR)

In [4]:
reform = Calculator.read_json_param_objects('TCJA_House_Amended.json',
                                            'JCT_Behavior.json')
pol = Policy()
pol.implement_reform(reform['policy'])
con = Consumption()
con.update_consumption(reform['consumption'])
beh = Behavior()
beh.update_behavior(reform['behavior'])

In [5]:
def total_combined_liability(calc):
    # combined is combined tax liability while s006 is weight.
    return (calc.records.combined * calc.records.s006).sum()

In [6]:
def print_combined_liability(baseline_calc, reform_calc):
    # Calculate and print liabilities.
    baseline = total_combined_liability(baseline_calc)
    reformed = total_combined_liability(reform_calc)
    diff = reformed - baseline
    print ('Combined Liability - Baseline: {:0.2f}'.format(baseline))
    print ('Combined Liability - Reform: {:>18.2f}'.format(reformed))
    print ('-' * 47)
    print ('Difference: {:35.2f}'.format(diff))

In [7]:
calc_orig = Calculator(records=recs_orig, policy=Policy())
calc_orig.advance_to_year(2018)
calc_orig.calc_all()
calc_orig_reform = Calculator(records=recs_orig, policy=pol,
                              consumption=con, behavior=beh)
calc_orig_reform.advance_to_year(2018)
calc_orig_response = Behavior.response(calc_orig, calc_orig_reform)
print_combined_liability(calc_orig, calc_orig_response)

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
Combined Liability - Baseline: 2521070511449.64
Combined Liability - Reform:   2458101656483.74
-----------------------------------------------
Difference:                     -62968854965.90


In [8]:
calc_alt = Calculator(records=recs_alt, policy=Policy())
calc_alt.advance_to_year(2018)
calc_alt.calc_all()
calc_alt_reform = Calculator(records=recs_alt, policy=pol,
                              consumption=con, behavior=beh)
calc_alt_reform.advance_to_year(2018)
calc_alt_response = Behavior.response(calc_alt, calc_alt_reform)
print_combined_liability(calc_alt, calc_alt_response)

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
Combined Liability - Baseline: 2718451561189.63
Combined Liability - Reform:   2656473608389.99
-----------------------------------------------
Difference:                     -61977952799.63
